# Natural Language Processing Techniques on Public Records Requests Data

In [107]:
import spacy
import pandas as pd
import xlrd
#from sklearn.manifold import TSNE
from spacy.tokenizer import Tokenizer
from gensim.models import Word2Vec, ldamodel
from gensim.corpora import Dictionary
from collections import Counter
import itertools
from spacy.lang.en.stop_words import STOP_WORDS
import re
import numpy as np
import matplotlib.pyplot as plt
import pprint


# this isn't strictly an import, but it's used globally
pp = pprint.PrettyPrinter()

# change the model for different word vectors
nlp = spacy.load('en_core_web_lg')

# and finally, suppress an annoying warning that pandas throws
# when using straightforward indexing methods
pd.options.mode.chained_assignment = None

# Cleaning and structuring data for analysis

At the onset of this project, we obtained data extracts from a number of institutions and municipalities of different sizes, with the intention of collating them into one repository for analysis. Upon receipt, there were a handful of things that we realized, which dramatically reduced the kind of analysis that we could perform on the data.

1. **Government bodies have different reporting standards for metadata.** I believe this is related to the platform/infrastructure the institution is using to fulfill and manage requests. The fields of interest, as well as the types of data accessible within those fields, are often so distinct that they cannot be combined in a straightforward way. For example, some municipalities have tons of great metadata reporting, especially around the estimated amount of time and actual amount of time that it took to fulfill a request.

2. **Inputs are not standardized.** Both people and institutions submit public records requests, which means that there are differing degrees of detail, complexity, and formatting. Some people submit a few words or a case number in their request, while others copy-paste several spreadsheet columns. 

3. **There is personal identifying information everywhere.** The more I dug into records requests, the more I saw people signing their requests with their name, address, and contact information. It's hard to strip out this information with software, especially because requests ask for information on a particular person or named entity. This is problematic for open data projects, namely because 

4. **All data are equal, but some data are more equal than others.** This project charts out some methods that can be used across departments that have different numbers of records requests, but there's little we can do (and little inference we can provide) for a department that has but one solitary request.

The confluence of all of these factors effectively means that it's hard to look for trends. *Not great.*

It's really important to clean and harmonize the data as best we can. As such, we ended up throwing away most of the data that we had and working with a much smaller subset. After importing the data into a pandas dataframe, we kept:

- The department name (a string);
- Record creation date (a datetime object)
- Request summary (a string)

It's important to keep the orthography of the names the same, otherwise the information flow in the code here will break.

## Future Metadata Standards
Though we are only working with essentially three fields here, I would really like to see the following standard fields as metadata for each request:

- Unique Identifier for Record/Case (hash)
- Creation Date (datetime object)
- Assigned Department (string)
- Public Record Request (string; in a perfect world this would be plaintext, to prevent people from copy-pasting in spreadsheets)
- Close Date (datetime object)
- Attached Objects (myriad of formats)

In addition to those fields, these are some extensible fields that would be nice to see:

- Estimated Completion Date
- Actual Completion Date
- Type of Requester (controlled vocabulary; based on user research)

Notice that most of these fields are standard to the GovQA or WebQA platforms. If, through the Open Data Alliance, we were to shift to a 'data lake' model for collating and sharing this information, I would also like to see:

- Source Name (string; the jurisdiction from which the records request comes from)
- Source Type (controlled vocabulary, with possible choices like 'town', 'city', 'state', etc.)
- Tags (string; prepopulated based on text mining, and later expanded based on record officer input)

## Project 0: Cleaning the Data and Cursory Analysis
Let's load things up!

In [47]:
# this section loads an excel sheet.
# in an ideal scenario, all of these disparate fiels would live in a database with
# a set of standard fields
#
seadata = pd.read_excel('reformatted.xlsx')
olydata = pd.read_excel('StudentPRRLog_06212018_2.xlsx')
pordata = pd.read_excel('Port Orchard PRR Data.xls', header=9)

In [108]:
# reindex the data above, using only the columns we care about, for olympia
oly_trunc = olydata[['Assigned Dept', 'Create Date', 'Public Record Desired']]
oly_trunc.columns = ['department_name', 'create_date', 'request_summary']

# convert everything in the request_summary column to a spring, so no errors
# are thrown when doing natural language processing on data later
oly_trunc['request_summary'] = oly_trunc['request_summary'].astype(str)

# then drop all of the null values
oly_trunc = oly_trunc.dropna().reset_index()
del oly_trunc['index']

# and similar, but for port orchard
droplist = [column for column in pordata.columns if 'Unnamed' in column]
pordata.drop(droplist, axis=1, inplace=True)
por_trunc = pordata[['Assigned Dept', 'Create Date', 'Public Record Desired']]
por_trunc.columns = ['department_name', 'create_date', 'request_summary']
por_trunc['request_summary'] = por_trunc['request_summary'].astype(str)
por_trunc = por_trunc.dropna().reset_index()
del por_trunc['index']

# same, but for seattle
sea_trunc = seadata[['department_name', 'create_date', 'request_summary']]
sea_trunc['request_summary'] = sea_trunc['request_summary'].astype(str)
sea_trunc = sea_trunc.dropna().reset_index()
del sea_trunc['index']

# number of entries in the truncated data blocks
print("olympia data entries:", len(oly_trunc),
      "\nseattle data entries:", len(sea_trunc),
      "\nport orchard data entries:", len(por_trunc))
#sum(truncated.request_type_description != truncated.spd_overall_rec_req_description)

# later on, consider using NA values as test set for topic modeling. might be overkill?

olympia data entries: 7640 
seattle data entries: 23414 
port orchard data entries: 246


In [84]:
# here's a list of all of the possible departments we can choose from
# as well as the overall percentage that they make up
#

def dept_count_pct(df):
    depts = df['department_name'].value_counts()
    deptsdf = pd.DataFrame(depts)
    deptsdf['pct'] = (deptsdf['department_name'] / len(df)).round(5)
    return deptsdf

dept_count_pct(oly_trunc)

,department_name,pct
Olympia Police Department,5118,0.66990
Community Planning and Development,1435,0.18783
Multiple Departments,317,0.04149
Fire,253,0.03312
Administrative Services,183,0.02395
Public Works,169,0.02212
Human Resources,77,0.01008
Legal,43,0.00563
Executive,24,0.00314
Parks,18,0.00236


In [86]:
dept_count_pct(sea_trunc)

,department_name,pct
SPD,13789,0.58930
SFD,3976,0.16992
Site Administrator,939,0.04013
SCI,809,0.03457
FAS,740,0.03163
DOT,667,0.02851
LAW,297,0.01269
LEG,294,0.01256
SCL,279,0.01192
SPU,270,0.01154


In [87]:
dept_count_pct(por_trunc)

,department_name,pct
City Clerk's Office,183,0.75309
Community Development,45,0.18519
Public Works Department,14,0.05761
Finance Department,1,0.00412


Now that the data is clean and massaged into a form that we can easily work with, we should try to gather some descriptive information about records. Namely, how long are they, on average? We'll measure this in terms of characters, as well as tokens (words).

In [94]:
# this is a helper function that gives us the average length in characters
# for each request for each department, and then a higher-level descriptive table
# of the same
def avglen(df):
    # ARGUMENTS: df; a dataframe
    # OUTPUTS: a dict; containing column name, average len in chars, number of requests
    #
    
    # get a shortlist of all the names of departments
    names = set(df.department_name.values)
    
    # for 
    varlist = [df[df.department_name == i]['request_summary'] 
               for i in names]
    
    l = [len(i) for i in varlist]
    s = []    
    for i in varlist:
        s.append(sum([len(str(j)) for j in i]))
    
    avg = [i / j for i,j in zip(s, l)]
    
    lendict = {n : (v, e ) for n, v, e in zip(names, avg, l)}

    return pd.DataFrame.from_dict(lendict, orient='index',
                             columns=['average length (chars)', 'requests per dept'])

olyagl = avglen(oly_trunc)
olyagl

,average length (chars),requests per dept
Multiple Departments,436.861199,317
Administrative Services,250.065574,183
Human Resources,297.311688,77
Legal,170.395349,43
Fire,101.988142,253
Public Works,292.502959,169
Parks,241.333333,18
Municipal Court,53.000000,3
Community Planning and Development,177.438328,1435
Olympia Police Department,152.511333,5118


In [98]:
poragl = avglen(por_trunc)
poragl

,average length (chars),requests per dept
City Clerk's Office,388.480874,183
Community Development,267.333333,45
Finance Department,192.000000,1
Public Works Department,284.785714,14


In [102]:
seaagl = avglen(sea_trunc)
seaagl

,average length (chars),requests per dept
SPL,501.454545,11
FAS,423.393243,740
DON,504.879310,58
RET,435.828947,76
LEG,433.112245,294
OFH,647.030303,33
IMR,336.428571,7
SCL,503.150538,279
PPN,170.000000,1
OCR,302.913907,151


In [169]:
desc = {'seattle': seaagl.describe()[1:],
                'olympia': olyagl.describe()[1:],
                'port orchard' : poragl.describe()[1:]}

descriptions = pd.concat(desc)
descriptions

average length (chars)  requests per dept
olympia      mean              226.336325         694.545455
             std               109.130513        1522.565490
             min                53.000000           3.000000
             25%               161.453341          33.500000
             50%               241.333333         169.000000
             75%               294.907323         285.000000
             max               436.861199        5118.000000
port orchard mean              283.149980          60.750000
             std                80.943927          83.563848
             min               192.000000           1.000000
             25%               248.500000          10.750000
             50%               276.059524          29.500000
             75%               310.709504          79.500000
             max               388.480874         183.000000
seattle      mean              439.574102         632.405405
             std               226.484746        2322.359605
             min                16.053248           1.000000
             25%               350.169231          11.000000
             50%               435.828947          58.000000
             75%               504.879310         270.000000
             max              1374.833333       13789.000000

In [105]:
bydate = por_trunc.set_index('create_date')

bydate
por_trunc['summary_token'] = df.apply(lambda row: 
                                      nltk.word_tokenize(row['sentences']), axis=1)

def tokenizedf(df, col):
    # takes in a dataframe
    # tokenizes the columns in the dataframe
    # returns a dataframe with tokenized columns
    df = df['tokens']
    for row in df[col]:
        row = len([nlp_proc(str(row))])
        df.
    return df

test = tokenizedf(bydate, 'request_summary')

,department_name,request_summary
create_date,,
2016-06-10 22:38:00,City Clerk's Office,I am requesting a copy of any allegations of p...
2016-06-13 19:50:00,City Clerk's Office,"Regarding Police Officer Nathan Lynch #734, I ..."
2016-06-16 14:14:00,City Clerk's Office,test request for training
2016-06-17 11:32:00,City Clerk's Office,Matrix from staff based on our letter (provide...
2016-06-17 11:53:00,City Clerk's Office,Can you send me the power point on the road cl...
2016-06-17 12:18:00,Community Development,Copy of site plans for gas station at 390 SW S...
2016-06-23 12:31:00,City Clerk's Office,RE: Chris Tibbs' Application to Port Orchard C...
2016-06-23 12:32:00,City Clerk's Office,I would like to see a copy of the approved dra...
2016-06-23 20:39:00,City Clerk's Office,1) The mounting location of Officer Nathan Lyn...


# Project 1: Generating a word cloud for each department

Word clouds, while occasionally superficial, are handy proof-of-concept precursors for more complex linguistic analysis. In order to generate a word cloud, you have to go through most of the same processes (cleaning and splitting data, breaking into constituent parts, etc.) that you do when working on a machine learning pipeline, for example. To that end, here's a wall of code that is used to generate a word cloud.

In [138]:
def sentences(df, col):
    # ARGUMENTS: df; a pandas dataframe
    # OUTPUTS: col, a string; a column of a particular pandas dataframe
    #
    sents = []
    #
    # this function takes information out of a table
    # and dumps it into a list for further processing
    datesplit = re.compile('\d+|\D+')
    
    for row in df[col]:
        row = str(row)
        # some cleaning happens here as well, to make sure
        # that people who use more complex formatting (i.e.
        # unicode characters) have their information handled properly
        row = row.replace("\t", " ")
        row = row.replace("\p", " ")
        row = row.replace('\uf07f', " ")
        row = row.replace('\uf0b7', " ")
        row = row.replace('\uf071', " ")
        sents.append(str(row))
    return sents

def nlp_proc(data):
    # ARGUMENTS: data, either a string or a list
    # OUTPUTS: spacy processed text data
    #
    # this checks to see if the data we are processing is a list
    # and if it is, runs the NLP function on the different huge
    # strings in the list, returning the list
    if type(data) == list:
        nlplist = [ nlp(i) for i in data ]
        return nlplist
    else:
        # otherwise it just processes the data; just a backup
        return nlp(data)
    
def filter_noise(token):
    # ARGUMENTS: token; a spacy token
    # optional: mtl (minimum token length); an int
    # optional: cs (custom stop); a boolead
    # OUTPUTS: T/F
    #
    is_noise = False
    # this function performs a series of checks to see if 
    # a token is noise and then returns t/f
    # essentially it's a giant switch
    #
    #
    #
    # here's a regular expression for matching dates/times from a string
    # spacy doesn't handle that task well
    dates = re.compile('\d{1,2}(?P<sep>[-/])\d{1,2}(?P=sep)\d{2,4}')
    times = re.compile(r'\d{1,2}(:\d{1,2})?(am|pm)?')
    
    #
    # filters stop words
    if token.is_stop == True:
        is_noise = True
    elif token in STOP_WORDS:
        is_noise = True
        
    # measures length of token; default is 3
    elif len(token.text) <= 3:
        is_noise = True
    
    # regex filters
    elif bool(dates.findall(token.text)) == True:
        is_noise = True
    elif bool(times.findall(token.text)) == True:
        is_noise = True
    elif token.text == '-PRON-':
        is_noise = True
        
    # filters things that are/look like numbers
    elif token.is_digit == True:
        is_noise = True
    elif token.is_currency == True:
        is_noise = True
    elif token.like_num == True:
        is_noise = True
    # filters web stuff
    elif token.like_url == True:
        is_noise = True
    elif token.like_email == True:
        is_noise = True
        
    # filters punctuation
    elif token.is_punct == True:
        is_noise = True
    elif token.is_left_punct == True:
        is_noise = True
    elif token.is_right_punct == True:
        is_noise = True
    elif token.is_bracket == True:
        is_noise = True
    elif token.is_quote == True:
        is_noise = True
    elif token.is_space == True:
        is_noise = True
    elif token.is_alpha == False:
        is_noise = True
    return is_noise 


def lem_stop(text, l=True):
    # ARGUMENTS: text; a list containing lists, containing spacy tokens
    # l; a boolean flag, True by default
    # OUTPUTS: a list containing lists, containing lemmas of tokens
    #
    # empty list, later to become a list of lists
    txt = []
    #
    for sentences in text:
        if l == True:
            # if the default isn't changed,
            # check to see if a word is noise; if not, keep the lemma form in the list
            sent = [token.lemma_ for token in sentences if filter_noise(token) == False]
        else:
            # this alternative mode here is mostly for testing purposes
            # and is ignored in the pipeline function
            # 
            # if the defaults are different, just add the complete token
            sent = [token for token in sentences if filter_noise(token) == False] 
        # then append the sentence to the list
        txt.append(sent)
    return txt

# after we transform the text from raw strings into lemmas, we have
# to do a secondary layer of filtering to remove additional strings that we
# don't really care about, or things that don't contribute in a strong way
# to our analysis

def filter_lemmas(text, a=False):
    # ARGUMENTS: lemma; a unicode string
    # OPTIONAL FLAG: a; a boolean, defaults to False
    # OUTPUTS: T/F
    #
    #
    is_noise = False
    # these are some stop words that occur pretty frequently across docs;
    # it might make sense to expand these further
    custom_stop_words = ['this', 'that', 'please', 'be', 'file',
                         'copy', 'with', 'from', 'like',
                         'have', 'other', 'thank', 'and/or',
                        'with', '-PRON-']
    
    other_custom_stops = ['seattle', 'request', 'record', 'document',
                         'olympia', 'port', 'orchard']
    
    for lemma in text:
        if lemma in custom_stop_words:
            is_noise = True
        if a == True:
            if lemma in other_custom_stops:
                is_noise = True
    return is_noise

def rec(text, ner=False, vb=False):
    # ARGUMENTS: text; a list containing lists, containing spacy tokens
    # OUTPUTS: a list containing lists, containing named entities
    #
    # empty list, later to become a list of lists
    txt = []
    #
    for sentences in text:
        if ner == True:
            sent = [entity.text for entity in sentences.ents] 
            # then append the sentence to the list
        if vb == True:
            sent = [entity.text for entity in sentences if entity.pos_=='VERB'] 
        txt.append(sent)
    return txt

In [175]:
def pipeline(df, fl=False):
    # a wrapper for all of the functions above
    # input: df; a dataframe
    # output: export; a list
    #
    # Flags
    #
    # running this should take approximately 15min on the full seattle dataset
    #
    # ensures that we have a 'request' column to work on
    # if we don't, this won't work.
    
    assert 'request_summary' in df.columns
    #
    #
    # this line takes all the text in the request summary column
    # and extracts each entry into a list
    o = sentences(df, 'request_summary')
    
    # extract the information, producing a list of sentences (really, a list of tokens)
    docs = nlp_proc(o)

    # finally, we:
    # remove stopwords
    # remove punctuation
    # lemmatize (reduce to simplest form for purposes of similarity)
    texts = lem_stop(docs)


    # then we filter out the lemmatized stopwords again,
    # but change the flag depending on different degrees of
    # filtering
    if fl==True:
    # if this flag is switched, then filter the additional stopwords
        return [i for i in texts if filter_lemmas(i, a=True) == False]
    
    else:
        
        return [i for i in texts if filter_lemmas(i) == False]
    

def export(df, fn):
    # ARGUMENTS: df; a dataframe
    # fn; string, a filename
    #
    # take the file name and append CSV
    filename = fn+"-data.csv"
    # and then write it to the current directory
    df.to_csv(filename, encoding='utf-8')

In [170]:
# this bit of code runs the above pipeline for each of the different entries in the dataset

porpipe = pipeline(por_trunc, fl=True)
olypipe = porpipe = pipeline(oly_trunc, fl=True)
seapipe = pipeline(sea_trunc, fl=True)




# and this helper function generates a dataframe of counts for each word,
# as well as proportion

def countdf(df):
    counter = Counter(itertools.chain(*df))
    cdf = pd.DataFrame.from_dict(counter, orient='index', columns=['count'])
    
    # percentage term frequency compared to rest of document
    cdf['pct'] = cdf['count'] / sum(cdf['count'])

    # normalization to reweight the "importance" of a word, invariate to document size
    cdf['prp'] = np.log2(cdf['count'])

    cdf = cdf.sort_values(by='count', index=True, ascending=False)

    return cdf

porcount = countdf(porpipe)
olycount = countdf(olypipe)
seacount = countdf(seapipe)

In [188]:
# this bit of code concatenates all of the different counts into a dataframe
# and then exports it.

citycounts = {'seattle': seacount,
              'olympia': olycount,
              'port orchard' : porcount}

counts = pd.concat(citycounts)

# running this will overwrite the existing csv file.
export(counts, 'city_counts')

## Project 1.1: Frequency of Named Entities

In our previous sections, we had broken up the body of texts into one huge list of words. Something potentially more interesting would be looking for entites (i.e. nouns or noun phrases of note) that appear frequently within a records request.

Having this information might suggest items that could be proactively disclosed. For example, if it turns out that one of the most frequently named noun phrases is 'building permits', it might make sense to see which type of building permits are out there, and further if there is a particular class of them that can be easily released to the public.

Moreover, if this information is already available, a department recieving these requests might want to redirect users to the place where the information already lives.

In [130]:
def filter_ents(text):
    # ARGUMENTS: lemma; a unicode string
    # OUTPUTS: T/F
    #
    is_noise = False
    # these are some stop words that occur pretty frequently across docs;
    # it might make sense to expand these further
    remove_ents = [' ', '', str([])]
    for ent in text:
        if ent in remove_ents:
            is_noise = True
    return is_noise

ents = rec(docs, ner=True)
collapse_ents = [' '.join(e) for e in ents if filter_ents(e) == False]

edf = pd.DataFrame(ents)#, columns=['string'])
#ecounter = pd.DataFrame(edf.value_counts())
#collapse = [str(e) for e in ents for i in e]
#ecounter
#edf.values.reshape(edf.shape, 1)
edf

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,Port Orchard,Nathan Lynch - #,734,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Nathan Lynch,734,"June 10th, 2016",this day,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Matrix,the City Council,the Comprehensive Plan,City Council,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,390,SW Sedgwick Rd,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Chris Tibbs' Application,Port Orchard City Council,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,the Dekalb Pier Improvements Phase 2 project,early this year,Bruce,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,1,Nathan Lynch,734,2,Nathan Lynch,between the hours of 1200 and 1330 on 6/10/16,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Linda Simpson,the City of Port Orchard,"January 1, 2015",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# Project 2: Clustering using LDA

### What is Latent Dirichlet Allocation (LDA)?
Latent dirichlet allocation (LDA) is a tool for finding implicit relationships in a large body of text. The algorithm produces topics, which essentially are groupings of words that we--statistically speaking--expect to have some degree of association (represented through co-occurrence within a document). As such, topics are not explicitly related through semantics or knowledge content--it is through later inference that we understand the emergent higher-level categories.

For example, consider a body of text that contains keywords like 'China', 'black', 'white', 'spotted', 'Croatia', 'cute', and 'bamboo'. It could be the case that a subset of these words are explicitly (and exclusively) related to a particular category, while others occur within each category with about the same frequency distribution.

Using LDA, we are able to categorize all of these words into groups that make sense. The computer groups 'China', 'bamboo', 'black', 'white', and 'cute' together, and our human inference suggests *'panda'*. In contrast, if we see 'Croatia', 'spotted', 'black', 'white', and 'cute', in the computer output, we think *'dalmatian'*. It's okay that the different topics overlap--as in human languages, semantic meaning is often distributed across different words.

More complicated algorithms can be used to assign actual labels to topic categories.

In [ ]:
# Create Dictionary for port orchard
id2word = Dictionary(porpipe)

# Term Document Frequency
corpus = [id2word.doc2bow(txt) for txt in texts]

passes = 50
rs = 7
# how do we decide on a good number of topics
num_topics = 10

lda_40 = ldamodel.LdaModel(corpus, 
                        num_topics=num_topics, 
                        id2word = id2word, 
                        passes=passes, 
                        random_state=rs)

In [ ]:
pp.pprint(lda_40.print_topics())

# subtract the sum of all of the topic coefficients from 1 in order to get the most viable topic model

# Project 2: Cosine Similarity Using Word2Vec

### What is Word2Vec?

Word2vec is an algorithmic system used to produce word embeddings, which may need some explanation or unpacking. Think back to the LDA section of this document. The computer produced a clustering of words, and we used our associative human creativity to establish patterns and relationships between them. What if it were possible to determine the similarity or semantic relationship between words programmatically?

Word2vec achieves this by taking a large body of text and representing it as a vector space. Each word contained within that vector space is encoded as a vector, comprised of a 1 where the word is and 0's everywhere else). There is a hidden filter layer which compresses the size of this vector while minimizing information loss, as smaller vectors are less computationally complex to compare. Finally, the word vectors are all positioned in the vector space relative to each other, with more similar words clustered together.

This is quite abstract, so let's try out an example. Assume that we have the sentences "Bananas and apples are delicious," and "Durian and jackfruit are unpleasant." We can represent each of these as a list of words:

`document1 = ['Bananas', 'and', 'apples', 'are', 'delicious', '.']`

`document2 = ['Durian', 'and', 'jackfruit', 'are', 'unpleasant', '.']`

And then as a vector space, like so:

In [87]:
d = {'bananas': [1, 0], 'durian': [0, 1], 'and': [1, 1],
     'apples': [1, 0], 'jackfruit': [0, 1], 'are': [1, 1],
    'delicious': [1, 0], 'unpleasant': [0, 1], '.': [1, 1]}

s  = pd.Series(d,index=d.keys())
s

bananas       [1, 0]
durian        [0, 1]
and           [1, 1]
apples        [1, 0]
jackfruit     [0, 1]
are           [1, 1]
delicious     [1, 0]
unpleasant    [0, 1]
.             [1, 1]
dtype: object

While we have produced an excellent vector space here, we can make the vector space more sparse and easier to work with by dropping items that do not have semantic relevance. That includes words like 'and' and punctuation.

In [83]:
e = {'bananas': [1, 0], 'durian': [0, 1],
     'apples': [1, 0], 'jackfruit': [0, 1],
    'delicious': [1, 0], 'unpleasant': [0, 1]}

t  = pd.Series(e,index=e.keys())
t

bananas       [1, 0]
durian        [0, 1]
apples        [1, 0]
jackfruit     [0, 1]
delicious     [1, 0]
unpleasant    [0, 1]
dtype: object

Again, this is our vector *space*. A *word vector* is the representation of the word with regard to the entire vecor space. The following word vectors are represented like this, relative to their presence within the vector space, and the documents in which they appear:

`bananas = [1, 0, 0, 0, 0, 0]`

`durian = [0, 1, 0, 0, 0, 0]`

`delicious = [0, 0, 0, 0, 1, 0]`

`unpleasant = [0, 0, 0, 0, 0, 1]`

This is a simplistic picture of how a word vector operates. There is little insight that we can derive from this, other than comparing direct equivalence of vectors. But things become interesting when there is a significantly large corpus of documents, with different uses and contexts for words.

Word2vec takes word vectors for every word that appears in a corpus (as above) and represents their contexts as a series of weights. Think of it like creating a dictionary, where each definition is composed of a little piece of every other definition, but to varying degrees. Because each definition of a word is created relationally, it is possible to capture conceptual or syntactic meaning an a really robust, fascinating (almost surprising) way.

Supposing we had more data (a huge set of other documents), the previous word vectors might be transformed to contain all of the "definitions" of the other words within the dataset:

`bananas = [0.89123, 0.66545, 0.19842, 0.11901, 0.09113, 0.07221]`

`durian = [0.12311, 0.71834, 0.22142, 0.13452, 0.08721, 0.067881]`

`delicious = [0.13317, 0.17004, 0.21891, 0.66311, 0.88313, 0.70019]`

`unpleasant = [0.14141, 0.16167, 0.22212, 0.57719, 0.77311, 0.87123]`

Other algorithms can be used to reduce the dimensionality of the vector space, such that each of these vectors can be plotted in 2D space. For a naive explanation of how this works, check out the bolded components of each of these vectors:

`bananas = [`**0.89123`, `0.66545**`, 0.19842, 0.11901, 0.09113, 0.07221]`

`durian = [`**0.71311`, `0.86834**`, 0.22142, 0.13452, 0.08721, 0.067881]`

`delicious = [0.13317, 0.17004, 0.21891, 0.38311, `**0.88313`, `0.70019**`]`

`unpleasant = [0.14141, 0.16167, 0.22212, 0.57719, `**0.77311`, `0.87123**`]`

In [45]:
# word2vec
#
# error messages of note, in case further problems arise:
# https://stackoverflow.com/questions/33989826/python-gensim-runtimeerror-you-must-first-build-vocabulary-before-training-th/33991111
# 

# the number of dimensions of generated vectors. this is a good number to
# play around with. some people suggest square-root length of vocabulary
# conceptually this might map onto principle components, or number of topics
size = 50

# terms that occur less than min_count number 
# of times are ignored in calculations
# may want to change this depending on reimplementation of
# lem_stop function above
min_count = 1

# terms that occur within this window of text are associated with it
# during the training of the model. if the corpus of text contains large
# sentences then it may be a good idea to change this to something larger.
# the documentation suggests 10 as an upper bound and 4-7 as a good range.
window = 4

# skip-gram technique: boolean that determines skipgram vs continuous bag of words
# model. the default is 1, skipgram
sg = 1

prr2vec = Word2Vec(
    [prr2vec_data],
    sg=sg,
    size=size,
    min_count=min_count,
    window=window
)

In [48]:
#prr2vec.wv['zone']
#prr2vec.wv.most_similar('state')
#prr2vec.wv.vocab


#prr2vec_data = [ lem_stop(i) for i in docs ]
# the output here is a list of all of the processed, filtered text
#
# this is wrapped in a list in order to help word2vec work with the word embeddings
# rather than just the characters of the text. normally word2vec takes sentences
# rather than just words

[('sincerely', 0.9992898106575012),
 ('act', 0.9992607235908508),
 ('category', 0.9992291331291199),
 ('which', 0.9991952180862427),
 ('each', 0.9991693496704102),
 ("'s", 0.9991583228111267),
 ('khandelwal@kingcounty', 0.9991531372070312),
 ('boise', 0.9991462230682373),
 ('be', 0.9991364479064941),
 ('in', 0.9991249442100525)]

In [129]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [130]:
tsne_plot(prr2vec)

/usr/local/anaconda3/envs/spacy/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


NameError: name 'TSNE' is not defined

#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [197]:
prr2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=2,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [208]:
prr2vec.build_vocab(docs.text)

token_count = sum([len(doc) for doc in docs.text])
print("The corpus contains {0:,} tokens".format(token_count))

RuntimeError: cannot sort vocabulary after model weights already initialized.

In [214]:
prr2vec.train(docs.text, total_examples=prr2vec.corpus_count, epochs=prr2vec.epochs)

ValueError: You must specify an explict epochs count. The usual value is epochs=model.epochs.

In [8]:
#define some parameters  
noisy_pos_tags = ['PROP']
min_token_length = 2

#Function to check if the token is a noise or not  
def isNoise(token):     
    is_noise = False
    if token.pos_ in noisy_pos_tags:
        is_noise = True 
    elif token.is_stop == True:
        is_noise = True
    elif len(token.string) <= min_token_length:
        is_noise = True
    return is_noise 

def cleanup(token, lower = True):
    if lower:
       token = token.lower()
    return token.strip()

cleaned_list = [cleanup(word.string) for word in docs if not isNoise(word)]
cleaned_list

['requesting',
 'copies',
 'of',
 'hsd',
 "'s",
 'service',
 'agreements',
 'with',
 'compass',
 'housing',
 'alliance',
 'and',
 'solid',
 'ground',
 'as',
 'well',
 'as',
 'correspondence',
 'emails',
 'and',
 'other',
 'documents',
 'that',
 'reference',
 'beatrice',
 'holbert',
 'carolyn',
 'kinniebrow',
 'or',
 'carolyn',
 'bilal',
 'reports',
 'regarding',
 'financial',
 'reviews',
 'of',
 'share',
 'in',
 '2009',
 '2011',
 'and',
 '2013from',
 'september',
 '2010',
 'all',
 'email',
 'correspondence',
 'from',
 'anyone',
 'at',
 'hsd',
 'that',
 'include',
 'any',
 'of',
 'the',
 'terms',
 'below',
 'peggy',
 'hotes”“scott',
 'morrow',
 'nate',
 'martin',
 'share',
 'wheel',
 'seattle',
 'housing',
 'and',
 'resource',
 'effort"“marvin',
 'futrell”"low',
 'income',
 'housing',
 'institute""michelle',
 'marchand""steven',
 'isaacson""lantz',
 'rowland',
 'jarvis',
 'capucion',
 'nickelsville""sharon',
 'lee""tent',
 'city"(1',
 'documents',
 'and',
 'communications',
 'that',
 'w

In [8]:
# str x in lambda to ensure that everything is processable by spacy
#civ['proc'] = civ['request_summary'].apply(lambda x: nlp(str(x)))
#listy = civ.proc.tolist()
s = ''
for row in civ['request_summary']:
    s += str(row)
    
doc = nlp(s)
tokenizer = Tokenizer(doc.vocab)

#for sentence in enumerate(text.sents):
#    print(sentence)
#    print('')

words = [token.lemma_ for token in doc if token.is_stop != True and token.is_punct != True]

# noun tokens that arent stop words or punctuations
nouns = [token.lemma_ for token in doc if token.is_stop != True and token.is_punct != True and token.pos_ == "NOUN"]

# five most common tokens
word_freq = Counter(words)
common_words = word_freq.most_common(5)

# five most common noun tokens
noun_freq = Counter(nouns)
common_nouns = noun_freq.most_common(5)

In [49]:
tokens = []
lemma = []
pos = []

for doc in nlp.pipe(civ['request_summary'].astype('unicode').values, batch_size=50,
                        n_threads=3):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

civ['tokens'] = tokens
civ['lemma'] = lemma
civ['pos'] = pos

KeyboardInterrupt: 

In [ ]:
# ideas for analysis
is it possible to predict the department that a request goes to, given the body of a request?
https://towardsdatascience.com/machine-learning-for-text-classification-using-spacy-in-python-b276b4051a49

is it possible to do LDA or topic modeling, to gauge what people are typically writing about in PRRs?

is it possible to discover the frequency of a keyword over time, and plot it?

it it possible to collect n-grams, and uncover phrases of relevance?